In [ ]:
# スマメイトユーザーをTwitterのscreen_nameで検索し、レートを取得してマージ、出力する

In [3]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option("display.max_colwidth", 80)

In [36]:
# Twitterのscreen_nameを与えると、スマメイトのプレイヤー検索をして一番上に出てくるユーザーのマイページリンクを返す

def fetch_mypage_url(twitter_user_screen_name:str):
    time.sleep(1)
    search_url = f"https://smashmate.net/search/?search={twitter_user_screen_name}"
    html = requests.get(search_url)
    soup = BeautifulSoup(html.text,"html.parser")
    for tag in soup.find_all("a"):
        if "https://smashmate.net/user/" in tag.get('href'):
            return tag.get('href') # 検索結果一番上の1件だけを返す
    else:
        return "" # ヒット無しの場合空文字列を返す

In [49]:
# マイページURLを与えると、そのユーザーのレートを返す

def fetch_rate_from_mypage(mypage_url):
    if mypage_url == "":
        return ""
    
    time.sleep(1)
    
    html = requests.get(mypage_url)
    mypage_soup = BeautifulSoup(html.text,"html.parser")
    mypage_text = str(mypage_soup)
    record_text = mypage_text[mypage_text.find("<h2>レーティング対戦</h2>"):mypage_text.find("""<h2 class="mt-5">プロフィール</h2>""")] # 戦績だけ抽出
    record_text = BeautifulSoup(record_text,"html.parser").get_text(strip=True).replace("\t","") # 例:レーティング対戦今期レート1357 (12035位)前日比：-17今期対戦成績43勝 55敗現在2連勝！動画化許可する
	
    rate_idx = record_text.find("今期レート")
    if 0 < rate_idx: # 対戦記録があるとき
        return record_text[rate_idx+5:rate_idx+9]
    elif 0 < record_text.find("初期レート"): # サブシーズン0戦状態
        ini_rate_idx = record_text.find("初期レート")
        ini_rate = record_text[ini_rate_idx+5:ini_rate_idx+9] # 初期レート
        if ini_rate == "1500": # 大半が前期0戦。たまたま1500ぴったりの場合もあるが除く
            return ""
        else:
            return ini_rate
    else:
        return ""

In [54]:
# スマブラー格付けチェックに参加したユーザーのレートを取得し、出力する

df = (pd.read_csv("df_vote.csv", encoding="utf-8-sig")
      .assign(mypage_url = lambda x: x.twitter_user_screen_name.apply(fetch_mypage_url))
      .assign(smamate_rate = lambda x: x.mypage_url.apply(fetch_rate_from_mypage))
     )
df.to_csv("df_vote_and_rate.csv", index=False, encoding="utf-8-sig")
df

,vote,tw_time,twitter_user_id,twitter_user_name,twitter_user_screen_name,twitter_user_followers_count,vote_url,is_quote,twitter_user_prof,main,delta_hours,mypage_url,smamate_rate
0,A,2021-12-29 18:12:42+09:00,995831063392829440,神祖覇天,FERoyThany,827,twitter.com/FERoyThany/status/1476118971724480512,False,SSBU: Byleth/ミラー世界最弱/最近エアライド熱再燃してます/金鹿の学級担任/アイコン→らむだん5cさん(@lambdanN5c)/ヘッダー→...,other,0,,
1,B,2021-12-29 18:13:25+09:00,400069662,とら(ケンタロスでした),tiger_1211,62,twitter.com/tiger_1211/status/1476119150208892930,False,某氏の動画で紹介されていた構築が公開終了されていても声掛けいただければもっかい公開します 質問箱→https://t.co/rZFtv0O2MF you...,other,0,https://smashmate.net/user/59069/,
2,B,2021-12-29 18:13:38+09:00,1252562177359138817,ペケ,peke_pcg,51,twitter.com/peke_pcg/status/1476119207893159942,False,PTCGO,other,0,,
3,A,2021-12-29 18:15:11+09:00,1109679787126087681,G&W|ヨセフ@ゲッチ,andoyosefGW,293,twitter.com/andoyosefGW/status/1476119595505586177,False,スマブラ垢です\nオフラインは本当に当分厳しそう,other,0,https://smashmate.net/user/43399/,1468
4,B,2021-12-29 18:15:14+09:00,1344964453415043072,にそじそ,Real_Ninjin,250,twitter.com/Real_Ninjin/status/1476119607325130754,False,スマブラ2000 ユナイト1850,other,0,https://smashmate.net/user/78651/,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,A,2021-12-30 17:56:54+09:00,1371813202497925124,ちょん/Chon,Chon62389671,1346,twitter.com/Chon62389671/status/1476477383029190658,False,ちょん鯖に入りたい人はDMまでお願いします\nピカチュウ│ホムヒカ│セフィロス│ロイ\nスマブラSPオフライン勢\nポケモンユナイト（マスターレート16...,other,23,https://smashmate.net/user/87154/,1622
1865,A,2021-12-30 17:57:27+09:00,2231279402,あこ,Ak0sandaze,950,twitter.com/Ak0sandaze/status/1476477522846306305,False,生きてます,other,23,https://smashmate.net/user/886/,1546
1866,B,2021-12-30 17:57:45+09:00,940477989082472448,じぇりぃ,jerry_raramagi,125,twitter.com/jerry_raramagi/status/1476477599107153920,False,出雲霞が好き\n\nららマジ スマブラ ウマ娘 osu! が好き\n最近は原神にドハマりしてる人になっている\n\nアイコンの描いて下さった神様(@_1...,other,23,https://smashmate.net/user/41121/,
1867,A,2021-12-30 17:58:28+09:00,2890454023,kept/Rush Gaming,FaintKept,45506,twitter.com/FaintKept/status/1476477777922891777,False,EPI主催\n\nスマブラSP日本ランク16位\nスマブラを軸に色々なことを楽しんでます\n@RushGamingJP,other,23,https://smashmate.net/user/302/,1595
